# Imaging and deconvolution demonstration

This script makes a fake data set and then deconvolves it. Finally the full and residual visibility are plotted.

In [ ]:
%matplotlib inline

import os
import sys
import multiprocessing

sys.path.append(os.path.join('..', '..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.data.polarisation import PolarisationFrame
from arl.visibility.base import create_visibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits, smooth_image
from arl.image.deconvolution import deconvolve_cube, restore_cube
from arl.image.iterators import raster_iter
from arl.image.solvers import solve_image
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_low_test_image_composite, \
    create_low_test_beam
from arl.imaging import *
from arl.imaging.weighting import weight_visibility

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

Construct LOW configuration

In [ ]:
low = create_named_configuration('LOWBD2-CORE')

We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [ ]:
config = 'core'
if config == 'full':
    low = create_named_configuration('LOWBD2')
    b = 8e4
    cellsize = 0.00001
    npixel=10 * 2048
    padding = 1
    invert = invert_2d
    predict = predict_2d

else:
    low = create_named_configuration('LOWBD2-CORE')
    b = 4e3
    cellsize = 0.001
    npixel=256
    padding = 2
    invert = invert_wprojection
    predict = predict_wprojection

    
oversampling = 32

frequency = numpy.linspace(0.99e8, 1.01e8, 3)
channel_bandwidth=numpy.array([1e6, 1e6, 1e6])
times = numpy.linspace(-3, +3, 15) * numpy.pi / 12.0
log.info('Observing times %s' % (times))

log.info("Observing frequencies %s Hz" % (frequency))

log.info("Cellsize = %.6f radians" % (cellsize))

In [ ]:
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')
vt = create_visibility(low, times, frequency, channel_bandwidth=channel_bandwidth,
                       weight=1.0, phasecentre=phasecentre, polarisation_frame=PolarisationFrame('stokesI'))

Plot the synthesized uv coverage, including for MFS

In [ ]:
plt.clf()
plt.plot(vt.uvw[:,0],   vt.uvw[:,1], '.', color='b')
plt.plot(-vt.uvw[:,0], -vt.uvw[:,1], '.', color='b')
plt.xlabel("U (wavelengths)")
plt.ylabel("V (wavelengths)")
plt.show()

Make a test image

In [ ]:
model = create_low_test_image_composite(npixel=npixel, frequency=frequency, channel_bandwidth=channel_bandwidth,
                                         cellsize=cellsize, 
                                         phasecentre=phasecentre)
export_image_to_fits(model, '%s/imaging-low-model.fits' % (results_dir))

In [ ]:
beam=create_low_test_beam(model)
model.data*=beam.data
print("Model * beam has %.3f Jy" % (numpy.sum(model.data[0,0,:,:])))
cmodel = smooth_image(model)
show_image(cmodel)
plt.title("Smoothed model image")
plt.show()
export_image_to_fits(cmodel, '%s/imaging-low-cmodel.fits' % (results_dir))
beam = None
cmodel = None

In [ ]:
vt.data['vis'] *= 0.0
vt = predict(vt, model, padding=1)

# To check that we got the prediction right, plot the amplitude of the visibility.
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis']), '.')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()

From now on, we will work with MFS images

In [ ]:
model = create_image_from_visibility(vt, npixel=npixel, nchan=1, phasecentre=phasecentre, 
                                     cellsize=cellsize)

Weight the data

In [ ]:
vt, density, densitygrid = weight_visibility(vt, model)
plt.clf()
plt.semilogy(uvdist, density, '.')
plt.xlabel('uvdist')
plt.ylabel('Sample density')
plt.show()
density = None
densitygrid = None

Make the dirty image and point spread function

In [ ]:
dirty, sumwt = invert(vt, model, padding=1)
show_image(dirty)

psf, sumwt = invert(vt, model, dopsf=True, padding=1)


print("Max, min in dirty image = %.6f, %.6f, sumwt = %s" % (dirty.data.max(), dirty.data.min(), sumwt))
print("Max, min in PSF         = %.6f, %.6f, sumwt = %s" % (psf.data.max(), psf.data.min(), sumwt))

export_image_to_fits(dirty, '%s/imaging-low-dirty.fits' % (results_dir))
export_image_to_fits(psf, '%s/imaging-low-psf.fits' % (results_dir))

dirty = None

Do some major cycles

In [ ]:
model.data[...] = 0.0
vtres, comp, residual = solve_image(vt, model, niter=10000, fractional_threshold=0.1, 
                                    threshold=0.050, nmajor=5, gain=0.1, 
                                    algorithm='hogbom', padding=padding,
                                    invert=invert, predict=predict)

fig=show_image(comp)
fig=show_image(residual)

In [ ]:
clean = restore_cube(model=comp, psf=psf, residual=residual)
export_image_to_fits(clean, '%s/imaging-low-clean.fits' % (results_dir))
show_image(clean)

Predict the visibility of the model

In [ ]:
vtmodel = create_visibility(low, times, frequency, channel_bandwidth=channel_bandwidth,
                            weight=1.0, phasecentre=phasecentre, 
                            polarisation_frame=PolarisationFrame('stokesI'))
vtmodel=predict(vtmodel, comp, timeslice=1.0)

Now we will plot the original visibility and the residual visibility.

In [ ]:
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis']), '.', color='b', label='Original')
plt.plot(uvdist, numpy.abs(vt.data['vis']-vtmodel.data['vis']), '.', color='r', 
         label='Residual')

plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.legend()
plt.show()